In [38]:
import numpy as np
import cv2
import time
import os 
from pathlib import Path
import re

def KeyPoints2CameraCoords(kp, K, w, h):
    # Keypoints to homogeneous point coordinates
    kp_tf = np.concatenate((kp, np.ones((np.shape(kp)[0],1))), axis=1)
    # Keypoints to camera coordinate system
    kp_tf = kp_tf @ np.linalg.inv(K.T)
    limits = np.squeeze(np.array([w, h, 1]) @ np.linalg.inv(K.T)) # e.g. [1920,1080,1] to camera coords
    w_k, h_k = limits[0,0], limits[0,1]
    # Define T_norm based on width and height of the image (now in camera coords)
    T_norm = np.diag([2/(w_k - 1.), 2/(h_k - 1.), 1]) # normalize
    T_norm[:,2] = [-1, -1, 1] # translate
    # Conditioning/Normalization of keypoints
    kp_tf = kp_tf @ T_norm.T
    return kp_tf, T_norm


def NormalizeKeypoints(kp, w, h):
    # Keypoints to homogeneous point coordinates
    kp_tf = np.concatenate((kp, np.ones((np.shape(kp)[0],1))), axis=1)
    # Define T_norm based on width and height of the image (now in camera coords)
    T_norm = np.diag([2/(w - 1.), 2/(h - 1.), 1]) # normalize
    T_norm[:,2] = [-1, -1, 1] # translate
    # Conditioning/Normalization of keypoints
    kp_tf = kp_tf @ T_norm.T
    return kp_tf, T_norm



    
class FeatureExtractor:
    def __init__(self):
        self.extractor = cv2.SIFT_create()
        
    def compute_features(self, img):
        pts = cv2.goodFeaturesToTrack(np.mean(img, axis=2).astype(np.uint8), 3000, qualityLevel=0.01, minDistance=7)
        kps = [cv2.KeyPoint(x=f[0][0], y=f[0][1], size=20) for f in pts]
        kp, des = self.extractor.compute(img, kps)
        return kp, des
        
        #kp, des = self.extractor.detectAndCompute(img,None)
        #return kp, des


class FeatureMatcher:
    def __init__(self):
        self.matcher = cv2.BFMatcher()
    def match_features(self, frame_prev, frame_cur):
        kp1, desc1 = frame_prev.keypoints, frame_prev.features
        kp2, desc2 = frame_cur.keypoints, frame_cur.features
        # Match descriptors.
        matches = self.matcher.knnMatch(desc1,desc2,k=1)
        # Sort the matches according to nearest neighbor distance ratio (NNDR) (CV course, exercise 4)
        distmat = np.dot(desc1, desc2.T)
        X_terms = np.expand_dims(np.diag(np.dot(desc1, desc1.T)), axis=1)
        X_terms = np.tile(X_terms,(1,desc2.shape[0]))
        Y_terms = np.expand_dims(np.diag(np.dot(desc2, desc2.T)), axis=0)
        Y_terms = np.tile(Y_terms,(desc1.shape[0],1))
        distmat = np.sqrt(Y_terms + X_terms - 2*distmat)
        ## We determine the mutually nearest neighbors
        dist1 = np.amin(distmat, axis=1)
        ids1 = np.argmin(distmat, axis=1)
        dist2 = np.amin(distmat, axis=0)
        ids2 = np.argmin(distmat, axis=0)
        pairs = []
        for k in range(ids1.size):
            if k == ids2[ids1[k]]:
                pairs.append(np.array([k, ids1[k], dist1[k]]))
        pairs = np.array(pairs)
        # We sort the mutually nearest neighbors based on the nearest neighbor distance ratio
        NNDR = []
        for k,ids1_k,dist1_k in pairs:
            r_k = np.sort(distmat[int(k),:])
            nndr = r_k[0]/r_k[1]
            NNDR.append(nndr)

        id_nnd = np.argsort(NNDR)
        return np.array(matches)[id_nnd]

class TransformationEstimator:
    # Uses ransac algorithm to find best estimation of essential matrix between images
    def __init__(self, N=10, Threshold=0.2):
        self.N_iterations = N
        self.Threshold = Threshold
        
    def F_from_point_pairs(xs,xss):
        # xs, xss: Nx3 homologous point coordinates, N>7
        # returns F: 3x3 Fundamental matrix
        # Coefficient matrix
        N = np.size(xs)[0]
        A = np.zeros((N, 9))
        for n in range(N):
            A[n,:] = np.kron(xss[n,:], xs[n,:])
        # Singular-value-decomposition
        U,D,V = np.linalg.svd(A, full_matrices=True, compute_uv=True)
        Fa = np.reshape(V[:,:,8], (3,3)) # approximation of F, could be > rank 2
        Ua,Da,Va = np.linalg.svd(Fa, full_matrices=True, compute_uv=True)
        F = (Ua * np.diag([Da[0,0], Da[1,1], 0])) @ Va
        return F 
    
    def E_from_point_pairs(xs,xss):
        # xs, xss: Nx3 homologous point coordinates, N>7
        # returns E: 3x3 Essential matrix
        # Coefficient matrix
        N = np.size(xs)[0]
        A = np.zeros((N, 9))
        for n in range(N):
            A[n,:] = np.kron(xss[n,:], xs[n,:])
        # Singular-value-decomposition
        U,D,V = np.linalg.svd(A, full_matrices=True, compute_uv=True)
        Ea = np.reshape(V[:,:,8], (3,3)) # approximation of F, could be > rank 2
        Ua,Da,Va = np.linalg.svd(Ea, full_matrices=True, compute_uv=True)
        E = (Ua * np.diag([1,1,0])) @ Va
        return E
      
    def ransack(self, cur_frame_kp, prev_frame_kp, matches, T_norm, K):
        # cur_frame_kp, prev_frame_kp: Nx2 pixel coordinates of keypoints matching between images
        # T_norm, 3x3 transformation matrix of keypoints: center of mass to (0,0), x and y to scale [-1,1]
        # K: camera calibration matrix
        N = np.size(cur_frame_kp)[0]
        highest_number_of_inliers = -1
        best_essential = np.eye(3)
        seed_n = 8 # has to be > 7
        # Keypoints to homogeneous point coordinates
        kp1 = np.concatenate((prev_frame_kp, np.ones(np.shape(N))))
        kp2 = np.concatenate((cur_frame_kp, np.ones(np.shape(N))))
        # Keypoints to camera coordinate system
        kp1 = kp1 @ K.T
        kp2 = kp2 @ K.T
        # Conditioning/Normalization of keypoints
        kp1 = kp1 @ T_norm.T
        kp2 = kp2 @ T_norm.T
        # Ransac loop
        for n in range(self.N_iterations):
            # Randomly select a seed group of > 7 matches
            seed_group = np.random.randint(low=0, high=N, size=seed_n)
            xs = kp1[seed_group,:]
            xss = kp2[seed_group,:]
            E = self.E_from_point_pairs(xs, xss)
        
        return(E)

class Frame:
    def __init__(self, rgb_fp, d_path, feature_extractor):
        self.rgb = cv2.imread(rgb_fp)
        self.depth = cv2.imread(d_path)
        self.keypoints, self.features  = None, None
        self.feature_extractor = feature_extractor
    def process_frame(self):
        self.keypoints, self.features = self.feature_extract(self.rgb)
        return self.keypoints, self.features, self.rgb
        
    def feature_extract(self, rgb):
        return self.feature_extractor.compute_features(rgb)
        
def compute_fundamental_matrix(kp1, kp2, matches):
    """
    Takes in filenames of two input images 
    Return Fundamental matrix computes 
    using 8 point algorithm
    """
    
    # extract points
    pts1 = []
    pts2 = []
    for i,(m) in enumerate(matches):
        #print(m.distance)
        pts2.append(kp2[m[0].trainIdx].pt)
        pts1.append(kp1[m[0].queryIdx].pt)
    pts1  = np.asarray(pts1)
    pts2 = np.asarray(pts2)
    
    # Compute fundamental matrix
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_8POINT)
    return F 

def essentialToRt(E):
    # see wikipedia: https://en.wikipedia.org/wiki/Essential_matrix
    U,d,Vt = np.linalg.svd(E)
    W = np.mat([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
    Winv = W.T
    # ansatz (educated guess)
    t = U @ W @ np.diag(d) @ U.T
    R = U @ Winv @ Vt
    return R, t

debug = True
scale = 5000
D = np.array([0, 0, 0, 0], dtype=np.float32)  # no distortion
K = np.matrix([[481.20, 0, 319.5], [0, 480.0, 239.5], [0, 0, 1]])  # camera intrinsic parameters
fx, fy, cx, cy = 481.20, 480.0, 319.5, 239.5

'''

def get_color(img, pt):
        x = int(np.clip(pt[0], 0, width - 1))
        y = int(np.clip(pt[1], 0, height - 1))
        color = img[y, x]
        if isinstance(color, Number):
            color = np.array([color, color, color])
        return color[::-1] / 255.

def point2dTo3d(n, m, d):
    z = float(d) / scale
    x = (n - cx) * z / fx
    y = (m - cy) * z / fy
    point = np.array([x, y, z], dtype=np.float32)
    return point


def solvePnP(frame1, frame2, matches):
        kp1, kp2, des1, des2, depth = frame1.keypoints, frame2.keypoints, frame1.features, frame2.features, frame1.depth
        goodMatches = matches
    
        pts_obj, pts_img2, pts_img1 = [], [], []
        colour = []
        features = []
        for i in range(0, len(goodMatches)):
            p = kp1[goodMatches[i][0].queryIdx].pt
            # d = depth[int(p[1])][int(p[0])]
            d = depth.getpixel((int(p[0]), int(p[1])))
            if d == 0:
                pass
            else:
                p2 = kp2[goodMatches[i][0].trainIdx].pt
                #dif = abs(cv2.norm(p) - cv2.norm(p2))
                #if dif > .1:
                    #print('dif -> {}'.format(dif))
                    #pass
                pts_img2.append(p2)
                pts_img1.append(p)
                pd = point2dTo3d(p[0], p[1], d)
                pts_obj.append(pd)
                # c = frame1.rgb[int(p[1])][int(p[0])]
                colour.append(get_color(img=frame1.rgb, pt=p))
                features.append(des1[goodMatches[i][0].queryIdx])

        pts_obj, pts_img2 = np.array(pts_obj), np.array(pts_img2)
        pts_img1 = np.array(pts_img1)
        if debug:
            print('pts_obj -> {}, pts_img->{}'.format(np.shape(pts_obj), np.shape(pts_img2)))
        retval, rvec, tvec, inliers = cv2.solvePnPRansac(pts_obj, pts_img2, K, D, useExtrinsicGuess=False)
    '''


def opencv_R_t(E,kp1, kp2, matches):
    """
    Takes in filenames of two input images 
    Return Fundamental matrix computes 
    using 8 point algorithm
    """
    
    # extract points
    pts1 = []
    pts2 = []
    for i,(m) in enumerate(matches):
        #print(m.distance)
        pts2.append(kp2[m[0].trainIdx].pt)
        pts1.append(kp1[m[0].queryIdx].pt)
    pts1  = np.asarray(pts1)
    pts2 = np.asarray(pts2)
    
    pts_l_norm = cv2.undistortPoints(np.expand_dims(pts1, axis=1), cameraMatrix=K, distCoeffs=None)
    pts_r_norm = cv2.undistortPoints(np.expand_dims(pts2, axis=1), cameraMatrix=K, distCoeffs=None)

    points, R, t, mask = cv2.recoverPose(E,pts_l_norm, pts_r_norm)
    return R,t

In [39]:
# Filepaths
import os
#cur_dir = os.getcwd()
cur_dir = "c:\\Users\\jerek\\Documents\\Koulu\\Aalto syksy 2022\\visual_slam\\"
dir_rgb = cur_dir + "/data/ICL_NUIM/rgb/"
dir_depth = cur_dir + "/data/ICL_NUIM/depth/"
is_WINDOWS = True
if is_WINDOWS:
    dir_rgb = dir_rgb.replace("/", "\\")
    dir_depth = dir_depth.replace("/", "\\")
# Initialize
print(dir_rgb)
feature_extractor = FeatureExtractor()
feature_matcher = FeatureMatcher()


# run feature extraction for 1st image
fp_rgb = dir_rgb + str(1) + ".png"
#print(fp_rgb)
im = cv2.imread(fp_rgb)
#print(im)
fp_depth = dir_depth + str(1) + ".png"
cur_frame = Frame(fp_rgb, fp_depth, feature_extractor)
kp1, features1, rgb1 = cur_frame.process_frame() 

prev_frame = cur_frame
i = 30 # jump to 30th frame

fp_rgb = dir_rgb + str(i) + ".png"
fp_depth = dir_depth + str(i) + ".png"
# Feature Extraction for current frame
cur_frame = Frame(fp_rgb, fp_depth, feature_extractor)
kp2, features2, rgb2 = cur_frame.process_frame()

# Feature Matching to previous frame
matches = feature_matcher.match_features(prev_frame, cur_frame) 

best8 = matches[:8]

F = compute_fundamental_matrix(kp1, kp2, matches[:100])
F
#img3 = cv2.drawMatchesKnn(prev_frame.rgb,prev_frame.keypoints, cur_frame.rgb,cur_frame.keypoints,matches[:8],None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#img2 = cv2.drawKeypoints(rgb, kp, None, color=(0,255,0), flags=0)
#cv2.imshow('a', img3)
#cv2.waitKey(0)
E = K.T @ F @ K
#print(E)

R, t = essentialToRt(E)
print(R)
print(t)

# E = U@np.diag(S)@VT
R, t = opencv_R_t(E,kp1, kp2,matches[:100])
print(R)
print(t)

#solvePnP()





c:\Users\jerek\Documents\Koulu\Aalto syksy 2022\visual_slam\\data\ICL_NUIM\rgb\
[[-9.99779581e-01  1.25375812e-02  1.68403866e-02]
 [-1.25463071e-02 -9.99921207e-01 -4.12599352e-04]
 [-1.68338867e-02  6.23793068e-04 -9.99858106e-01]]
[[ 5.19144418e-02 -6.26134151e+01  3.37150105e+00]
 [ 6.21205719e+01 -1.65189214e-01 -1.09700511e+02]
 [-3.52761128e+00  1.10589183e+02  1.13274773e-01]]
[[ 9.99779581e-01 -1.25375812e-02 -1.68403866e-02]
 [ 1.25463071e-02  9.99921207e-01  4.12599352e-04]
 [ 1.68338867e-02 -6.23793068e-04  9.99858106e-01]]
[[-0.86986325]
 [-0.02724269]
 [-0.49254011]]


In [43]:
vec = np.array([[-0.86986325], [-0.02724269],[-0.49254011]])
vec
np.linalg.norm(vec)

0.9999999989089053

In [ ]:
#print(np.array(kp1))

# extract points
pts1 = []
pts2 = []
for i,(m) in enumerate(matches):
    #print(m.distance)
    pts2.append(kp2[m[0].trainIdx])
    pts1.append(kp1[m[0].queryIdx])

pts1  = np.asarray(pts1)
pts2 = np.asarray(pts2)


F, mask = cv2.findFundamentalMat(pts1[:100], pts2[:100], cv2.FM_8POINT)
print("x2.T @ F @ x1 = (should be close to 0)" )


test_idx = 20

x1 = np.expand_dims(pts1[test_idx,:], axis=1)
x2 = np.expand_dims(pts2[test_idx,:], axis=1)


#print(np.shape(x2))



print(x2.T @ F @ x1)

E = K.T @ F @ K # Get the essential matrix

[U,S,V] = np.linalg.svd(E)

diag_110 = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 0]])
newE = U@diag_110@V
[U,S,V] = np.linalg.svd(newE); # Perform second decompose to get S=diag(1,1,0)

W = np.array([[0, -1, 0],[1, 0, 0],[0, 0, 1]]) # [0 -1 0; 1 0 0; 0 0 1];

R1 = U@W@V
R2 = U@W.T@V
t1 = U[:,2] #norm = 1
t2 = -U[:,2] #norm = 1


#print(R1, t1)


In [ ]:
T_norm = np.diag([2/(1920 - 1.), 2/(1080 - 1.), 1]) # normalize
T_norm[:,2] = [-1, -1, 1] # translate

a = np.array([1920,1080,1]) @ np.linalg.inv(K.T)

np.shape(np.squeeze(a))

In [ ]:
print([m[0].queryIdx for m in best8])
[m[0].trainIdx for m in best8]